![Curso Schwarz-Sosa-Suriano](http://www.fi.uba.ar/sites/default/files/logo.png)
# Sistemas de Ecuaciones Lineales
## Métodos Iterativos Estacionarios
***
**Curso Schwarz - Sosa - Suriano**
- Métodos Numéricos. *Curso 2*
- Análisis Numérico I. *Curso 4*
- Métodos Matemáticos y Numéricos. *Curso 6*

### Métodos iterativos:

A la hora de resolver sistemas de ecuaciones lineales nos encontraremos muchas veces con *matrices ralas*, caracterizadas por tener una gran cantidad de coeficientes nulos.

Si bien es cierto que en los métodos directos podemos estimar de antemano la cantidad de operaciones que debemos realizar para encontrar la solución, la factorización de matrices ralas suele devolver matrices L y U (o S) que tienen una mayor proporción de coeficientes no nulos que la matriz A original

Pensando en este tipo de matrices se han desarrollado los denominados métodos iterativos, en los que se parte de una solución inicial que se va corrigiendo iteración tras iteración. Para no realizar infinitas repeticiones se incorporan -de manera análoga a lo visto para las ecuaciones no lineales- criterios de corte basados en una tolerancia prefijada.

### Metodo de Jacobi:

$$x_i^{(k+1)} = \frac{b_i-\sum_{j=1}^{i-1}{a_{i,j}\cdot x_j^{(k)}} - \sum_{j=i+1}^{n}{a_{i,j}\cdot x_j^{(k)}}}{a_{i,i}}$$ 

Si la matriz A de nuestro SEL es estrictamente diagonal dominante, entonces la convergencia de Jacobi está garantizada para cualquier vector inicial, por lo que podemos elegir uno en forma arbitraria:
$$X^{(0)} = \begin{bmatrix} x_0^{(0)} \\ x_1^{(0)} \\ ... \\ x_n^{(0)}\end{bmatrix}$$

Para decidir cuándo debemos dejar de iterar es necesario incorporar al algoritmo un criterio de corte.

### Criterios de Corte:

Las siguientes son algunas de las condiciones que se podrían evaluar al final de cada iteración para decidir si se continúa iterando: 

$$ (1) \hspace{2mm} || R^{(k+1)} || = || B - A.X^{(k+1)}|| \leq TOL  \hspace{20mm} (2) \hspace{2mm} \frac{||R^{(k+1)}||}{||B||} \leq TOL$$

$$ (3) \hspace{2mm} || X^{(k+1)} - X^{(k)} || \leq TOL  \hspace{20mm}  (4) \hspace{2mm} \frac{|| X^{(k+1)} - X^{(k)} ||}{|| X^{(k+1)} ||} \leq TOL $$

Los criterios (1) y (2) se basan en analizar el residuo, y por supuesto el segundo de ellos debería ser considerado como el más apropiado, ya qe se plantea en términos relativos. No obstante, veremos que el residuo no siempre es una magnitud que deba calcularse necesariamente.

Los criterios (3) y (4) se basan en la comparación entre iteraciones, y obviamente preferiremos este último por estar planteado en términos relativos. Como este criterio aprovecha los valores de las aproximaciones de $X$, resultará en general de fácil aplicación. 

### Problema a resolver:

In [4]:
from IPython.display import display, Math
import numpy as np
A = np.matrix([[10,-1,2,0],[-1,11,-1,3],[2,-1,10,5],[0,3,5,20]])
print(A)
B = np.array([6,25,-11,15])

[[10 -1  2  0]
 [-1 11 -1  3]
 [ 2 -1 10  5]
 [ 0  3  5 20]]


En el bloque de código encontrarán una función `resolverJacobi(A,B,tol,pasos)` que permite resolver un SEL a partir de conocer la matriz $ A $ y el vector $ B $, fijar una tolerancia `tol` e indicar con `True` o `False` si queremos o no que se acompañe la resolución con el procedimiento paso a paso. 

Y si quieren obtener un código más limpio, sin ninguna referencia de paso a paso, pueden eliminar cada uno de los bloques que comienzan con `if (pasos):`

In [5]:
def resolverJacobi(A, B, tol, pasos=True):
    n = np.shape(A)[0]
    X0 = np.zeros(n)
    X1 = np.zeros(n)
    iteracion = 0
    dif = 2*tol
    while (dif>tol and iteracion<100):
        if (pasos):
            display(Math("\\text{Vamos a realizar la "+str(iteracion+1)+"ª iteración}")) 
            if (iteracion==0):
                 display(Math("\\text{Adoptamos como vector inicial } X^{(0)} = "+r"\begin{bmatrix} 0 \\ 0 \\ ... \\ 0 \end{bmatrix}"))            
        for i in range (0,n):
            X1[i] = B[i]
            if (pasos):
                display(Math("\\text{Vamos a calcular la "+str(i+1)+"ª componente de } X^{("+str(iteracion+1)+")}"))  
                exp = "x_"+str(i+1)+"^{("+str(iteracion+1)+")} = \\frac{b_"+str(i+1)
                expN = "x_"+str(i+1)+"^{("+str(iteracion+1)+")} = \\frac{"+str(B[i])        
            for j in range(0,i):
                X1[i] -= A[i,j]*X0[j] 
                if (pasos):
                    exp += " - a_{"+str(i+1)+","+str(j+1)+"} \cdot x_"+str(j+1)+"^{("+str(iteracion)+")}"
                    expN += " - "+str(A[i,j])+"\cdot"+str(X0[j])
            for j in range(i+1,n):
                X1[i] -= A[i,j]*X0[j] 
                if (pasos):
                    exp += " - a_{"+str(i+1)+","+str(j+1)+"} \cdot x_"+str(j+1)+"^{("+str(iteracion)+")}"
                    expN += " - "+str(A[i,j])+"\cdot"+str(X0[j])                
            X1[i] /= A[i,i]
            if (pasos):
                exp += "}{a_{"+str(i+1)+","+str(i+1)+"}}"
                expN += "}{"+str(A[i,i])+"} = "+str(X1[i])
                display(Math(exp))
                display(Math(expN))  
        dif = np.linalg.norm(np.transpose(X1-X0),np.inf) / np.linalg.norm(X1.transpose(),np.inf)                
        if (pasos):   
            exp = "\\text{Evaluamos el criterio corte: } \\frac{|| X^{("+str(iteracion+1)+")} - X^{("+str(iteracion)+")} ||}{|| X^{("+str(iteracion+1)+")} ||} = "+str(dif)            
            if (dif>tol):
                exp += " > tol = "+str(tol)
                display(Math(exp)) 
            else:
                exp +=  " \leq tol = "+str(tol)
                display(Math(exp)) 
                exp = "\\text{Y hemos encontrado nuestra solución aproximada: } X^{("+str(iteracion+1)+")} = \\begin{bmatrix}"
                for i in range (0,n):
                    exp += str(X1[i])+r"\\"
                exp += r"\end{bmatrix}"
                display(Math(exp))
        X0 = X1.copy()
        iteracion += 1
    return X1 


In [6]:
resolverJacobi(A,B,10**-4)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

array([ 1.10812988,  2.00646679, -1.53754533,  0.83335311])

### Metodo de Gauss Seidel:

Analizando la expresión de Jacobi vemos que la primer sumatoria que aparece en el numerador usa valores de $X^{(k)}$ cuando podría estar usando los de $X^{(k+1)}$ que se van calculando en cada iteración. 

$$x_i^{(k+1)} = \frac{b_i-\sum_{j=1}^{i-1}{a_{i,j}\cdot x_j^{(k)}} - \sum_{j=i+1}^{n}{a_{i,j}\cdot x_j^{(k)}}}{a_{i,i}}$$

Si aplicamos esa modificación, llegamos al Método de Gauss Seidel: 

$$x_i^{(k+1)} = \frac{b_i-\sum_{j=1}^{i-1}{a_{i,j}\cdot x_j^{(k+1)}} - \sum_{j=i+1}^{n}{a_{i,j}\cdot x_j^{(k)}}}{a_{i,i}}$$


Al ser una mejora del método de Jacobi, también tenemos en este caso la convergencia asegurada si la matriz A es estrictamente diagonal dominante. 

No obstante, al ser un caso particular del método SOR, convergerá siempre que la matriz A sea definida positiva, lo que le da un rango de acción algo más amplio que el del método de Jacobi.

In [9]:
def resolverGaussSeidel(A, B, tol, pasos=True):
    n = np.shape(A)[0]
    X0 = np.zeros(n)
    X1 = np.zeros(n)
    iteracion = 0
    dif = 2*tol
    while (dif>tol and iteracion<100):
        if (pasos):
            display(Math("\\text{Vamos a realizar la "+str(iteracion+1)+"ª iteración}")) 
            if (iteracion==0):
                 display(Math("\\text{Adoptamos como vector inicial } X^{(0)} = "+r"\begin{bmatrix} 0 \\ 0 \\ ... \\ 0 \end{bmatrix}"))            
        for i in range (0,n):
            X1[i] = B[i]
            if (pasos):
                display(Math("\\text{Vamos a calcular la "+str(i+1)+"ª componente de } X^{("+str(iteracion+1)+")}"))  
                exp = "x_"+str(i+1)+"^{("+str(iteracion+1)+")} = \\frac{b_"+str(i+1)
                expN = "x_"+str(i+1)+"^{("+str(iteracion+1)+")} = \\frac{"+str(B[i])        
            for j in range(0,i):
                X1[i] -= A[i,j]*X1[j] 
                if (pasos):
                    exp += " - a_{"+str(i+1)+","+str(j+1)+"} \cdot x_"+str(j+1)+"^{("+str(iteracion+1)+")}"
                    expN += " - "+str(A[i,j])+"\cdot"+str(X1[j])
            for j in range(i+1,n):
                X1[i] -= A[i,j]*X0[j] 
                if (pasos):
                    exp += " - a_{"+str(i+1)+","+str(j+1)+"} \cdot x_"+str(j+1)+"^{("+str(iteracion)+")}"
                    expN += " - "+str(A[i,j])+"\cdot"+str(X0[j])                
            X1[i] /= A[i,i]
            if (pasos):
                exp += "}{a_{"+str(i+1)+","+str(i+1)+"}}"
                expN += "}{"+str(A[i,i])+"} = "+str(X1[i])
                display(Math(exp))
                display(Math(expN))  
        dif = np.linalg.norm(np.transpose(X1-X0),np.inf) / np.linalg.norm(X1.transpose(),np.inf)                
        if (pasos):   
            exp = "\\text{Evaluamos el criterio corte: } \\frac{|| X^{("+str(iteracion+1)+")} - X^{("+str(iteracion)+")} ||}{|| X^{("+str(iteracion+1)+")} ||} = "+str(dif)            
            if (dif>tol):
                exp += " > tol = "+str(tol)
                display(Math(exp)) 
            else:
                exp +=  " \leq tol = "+str(tol)
                display(Math(exp)) 
                exp = "\\text{Y hemos encontrado nuestra solución aproximada: } X^{("+str(iteracion+1)+")} = \\begin{bmatrix}"
                for i in range (0,n):
                    exp += str(X1[i])+r"\\"
                exp += r"\end{bmatrix}"
                display(Math(exp))
        X0 = X1.copy()
        iteracion += 1
    return X1 


In [10]:
resolverGaussSeidel(A,B,10**-4)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

array([ 1.10817603,  2.00637659, -1.53772479,  0.83347471])

### Metodo de las Sobrerrelajaciones Sucesivas (SOR):

Este método tiene su convergencia asegurada para cualquier valor de $0 < \omega < 2$ si la matriz A es definida positiva:

$$x_i^{(k+1)} = (1-\omega) \cdot x_i^{(k)} + \omega \cdot \frac{b_i-\sum_{j=1}^{i-1}{a_{i,j}\cdot x_j^{(k+1)}} - \sum_{j=i+1}^{n}{a_{i,j}\cdot x_j^{(k)}}}{a_{i,i}}$$
 
Es interesante observar que si $\omega = 1$ se convierte en el método de Gauss Seidel -por lo que aquel también converge si A es definida positiva.

En el desarrollo de este código, incorporamos un parámetro `w` para representar a $\omega$.

In [11]:
def resolverSOR(A, B, w, tol, pasos=True):
    n = np.shape(A)[0]
    X0 = np.zeros(n)
    X1 = np.zeros(n)
    iteracion = 0
    dif = 2*tol
    while (dif>tol and iteracion<100):
        if (pasos):
            display(Math("\\text{Vamos a realizar la "+str(iteracion+1)+"ª iteración}")) 
            if (iteracion==0):
                 display(Math("\\text{Adoptamos como vector inicial } X^{(0)} = "+r"\begin{bmatrix} 0 \\ 0 \\ ... \\ 0 \end{bmatrix}"))            
        for i in range (0,n):
            X1[i] = B[i]
            if (pasos):
                display(Math("\\text{Vamos a calcular la "+str(i+1)+"ª componente de } X^{("+str(iteracion+1)+")}"))  
                exp = "x_"+str(i+1)+"^{("+str(iteracion+1)+")} = (1-\\omega) \\cdot x_"+str(i+1)+"^{("+str(iteracion)+")} + \\omega \\cdot \\frac{b_"+str(i+1)
                expN = "x_"+str(i+1)+"^{("+str(iteracion+1)+")} = (1-"+str(w)+") \\cdot "+str(X0[i])+" + "+str(w)+" \\cdot \\frac{"+str(B[i])        
            for j in range(0,i):
                X1[i] -= A[i,j]*X1[j] 
                if (pasos):
                    exp += " - a_{"+str(i+1)+","+str(j+1)+"} \cdot x_"+str(j+1)+"^{("+str(iteracion+1)+")}"
                    expN += " - "+str(A[i,j])+"\cdot"+str(X1[j])
            for j in range(i+1,n):
                X1[i] -= A[i,j]*X0[j] 
                if (pasos):
                    exp += " - a_{"+str(i+1)+","+str(j+1)+"} \cdot x_"+str(j+1)+"^{("+str(iteracion)+")}"
                    expN += " - "+str(A[i,j])+"\cdot"+str(X0[j])                
            X1[i] /= A[i,i]
            X1[i] = (1-w) * X0[i] + w*X1[i]
            if (pasos):
                exp += "}{a_{"+str(i+1)+","+str(i+1)+"}}"
                expN += "}{"+str(A[i,i])+"} = "+str(X1[i])
                display(Math(exp))
                display(Math(expN))  
        dif = np.linalg.norm(np.transpose(X1-X0),np.inf) / np.linalg.norm(X1.transpose(),np.inf)                
        if (pasos):   
            exp = "\\text{Evaluamos el criterio corte: } \\frac{|| X^{("+str(iteracion+1)+")} - X^{("+str(iteracion)+")} ||}{|| X^{("+str(iteracion+1)+")} ||} = "+str(dif)            
            if (dif>tol):
                exp += " > tol = "+str(tol)
                display(Math(exp)) 
            else:
                exp +=  " \leq tol = "+str(tol)
                display(Math(exp)) 
                exp = "\\text{Y hemos encontrado nuestra solución aproximada: } X^{("+str(iteracion+1)+")} = \\begin{bmatrix}"
                for i in range (0,n):
                    exp += str(X1[i])+r"\\"
                exp += r"\end{bmatrix}"
                display(Math(exp))
        X0 = X1.copy()
        iteracion += 1
    return X1 


In [15]:
resolverSOR(A,B,1.1,10**-4)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

array([ 1.1081853 ,  2.00636719, -1.53773947,  0.83347981])